# 행안부 도로명주소 API 활용

In [1]:
import requests
from urllib.parse import quote

- API Key 가져오기

In [2]:
from google.colab import files
up = files.upload()

Saving roadapikey.txt to roadapikey.txt


In [3]:
with open('roadapikey.txt') as rf:
    road_key = rf.read()

- URL 만들기

In [4]:
# 한글은 반드시 인코딩 해주어야 함
quote('광주광역시청')

'%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C%EC%B2%AD'

In [5]:
# URL은 copy-paste
base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'currentPage=1&countPerPage=10&confmKey={road_key}'
params2 = f'keyword={quote("광주광역시청")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

- 요청해서 결과 얻기

In [6]:
result = requests.get(url)
result.status_code          # 정상적인 경우 200

200

In [8]:
result.text

'{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"1","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"111 Naebang-ro, Seo-gu, Gwangju","rn":"내방로","emdNm":"치평동","zipNo":"61945","roadAddrPart2":"(치평동)","emdNo":"04","sggNm":"서구","jibunAddr":"광주광역시 서구 치평동 1200 광주광역시청","siNm":"광주광역시","roadAddrPart1":"광주광역시 서구 내방로 111","bdNm":"광주광역시청","admCd":"2914012000","udrtYn":"0","lnbrMnnm":"1200","roadAddr":"광주광역시 서구 내방로 111(치평동)","lnbrSlno":"0","buldMnnm":"111","bdKdcd":"0","liNm":"","rnMgtSn":"291403160004","mtYn":"0","bdMgtSn":"2914012000112000000003487","buldSlno":"0"}]}}'

- 결과 해석하기: JSON --> Python 데이터 구조로 변환

In [7]:
import json
res = json.loads(result.text)
res

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '1',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '111 Naebang-ro, Seo-gu, Gwangju',
    'rn': '내방로',
    'emdNm': '치평동',
    'zipNo': '61945',
    'roadAddrPart2': '(치평동)',
    'emdNo': '04',
    'sggNm': '서구',
    'jibunAddr': '광주광역시 서구 치평동 1200 광주광역시청',
    'siNm': '광주광역시',
    'roadAddrPart1': '광주광역시 서구 내방로 111',
    'bdNm': '광주광역시청',
    'admCd': '2914012000',
    'udrtYn': '0',
    'lnbrMnnm': '1200',
    'roadAddr': '광주광역시 서구 내방로 111(치평동)',
    'lnbrSlno': '0',
    'buldMnnm': '111',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '291403160004',
    'mtYn': '0',
    'bdMgtSn': '2914012000112000000003487',
    'buldSlno': '0'}]}}

In [9]:
res.keys()

dict_keys(['results'])

In [10]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [11]:
len(res['results']['juso'])

1

In [12]:
res['results']['juso'][0].keys()

dict_keys(['detBdNmList', 'engAddr', 'rn', 'emdNm', 'zipNo', 'roadAddrPart2', 'emdNo', 'sggNm', 'jibunAddr', 'siNm', 'roadAddrPart1', 'bdNm', 'admCd', 'udrtYn', 'lnbrMnnm', 'roadAddr', 'lnbrSlno', 'buldMnnm', 'bdKdcd', 'liNm', 'rnMgtSn', 'mtYn', 'bdMgtSn', 'buldSlno'])

In [13]:
road_addr = res['results']['juso'][0]['roadAddr']
road_addr

'광주광역시 서구 내방로 111(치평동)'

- 광주광역시 주요 행정기관 주소

In [23]:
places = '광주광역시청,광주광역시 동구청,광주광역시 북구청,광주광역시 서구청,광주광역시 남구청,광산구청'.split(',')
places

['광주광역시청', '광주광역시 동구청', '광주광역시 북구청', '광주광역시 서구청', '광주광역시 남구청', '광산구청']

In [24]:
addr_list = []
for place in places:
    print(place)
    params2 = f'keyword={quote(place)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    res = json.loads(result.text)
    road_addr = res['results']['juso'][0]['roadAddr']
    addr_list.append(road_addr)

광주광역시청
광주광역시 동구청
광주광역시 북구청
광주광역시 서구청
광주광역시 남구청
광산구청


In [25]:
addr_list

['광주광역시 서구 내방로 111(치평동)',
 '광주광역시 동구 서남로 1(서석동)',
 '광주광역시 북구 우치로 77(용봉동)',
 '광주광역시 서구 경열로 33(농성동)',
 '광주광역시 남구 봉선로 1(봉선동)',
 '광주광역시 광산구 광산로29번길 15(송정동)']

- DataFrame으로 만들어 저장하기

In [26]:
import pandas as pd

df = pd.DataFrame({'장소':places, '도로명주소':addr_list})
df

,장소,도로명주소
0,광주광역시청,광주광역시 서구 내방로 111(치평동)
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동)
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동)
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동)
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동)
5,광산구청,광주광역시 광산구 광산로29번길 15(송정동)


In [27]:
df.to_csv('광주광역시내_주요기관.csv', index=False)